In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


In [2]:
from xgboost import XGBRegressor

# Read the data 
## Use group by to group by the total sales by department, saledate, and saletype

In [3]:
df = pd.read_csv('trnsact_final.csv')

In [4]:
df.head()

,Unnamed: 0,sku,store,city,state,zip,register,trannam,interid,saledate,...,amount,seq,mic,sale_month,saledayofweek,saleday,cost,retail,unknown,dept
0,1,3,4603,CORALVILLE,IA,52241,100,1200,0,2005-04-12,...,30.0,3400000,333,4,Tuesday,12,123.36,440.0,0,6505
1,2,3,802,CLEARWATER,FL,33761,660,400,0,2005-08-09,...,30.0,4700000,599,8,Tuesday,9,123.36,440.0,0,6505
2,3,3,2804,KNOXVILLE,TN,37919,360,5900,0,2005-05-09,...,30.0,2900000,202,5,Monday,9,123.36,440.0,0,6505
3,4,3,2102,ORLANDO,FL,32809,120,1700,0,2005-06-15,...,30.0,4600000,521,6,Wednesday,15,123.36,440.0,0,6505
4,5,3,2102,ORLANDO,FL,32809,950,800,0,2005-06-15,...,30.0,4700000,521,6,Wednesday,15,123.36,440.0,0,6505


In [5]:
df_by_dept = df.loc[:,['dept', 'amount', 'saledate', 'saletype']].groupby(['dept', 'saledate','saletype']).mean().reset_index()
df_by_dept['saledate'] = pd.to_datetime(df_by_dept['saledate'])

## Add the month, weekday, and day of month. Could add holiday or not here

In [6]:
df_by_dept['sale_month'] = df_by_dept['saledate'].dt.strftime('%m')
df_by_dept['sale_by_weekday'] = df_by_dept['saledate'].dt.strftime('%w')
df_by_dept['sale_by_day'] = df_by_dept['saledate'].dt.strftime('%d')

## Also calculate the cost and retail of the department and merge with previous one

In [7]:
cost_retail_dept = df.groupby('dept')[['cost','retail']].mean().reset_index()
df_by_dept = pd.merge(df_by_dept, cost_retail_dept, on = 'dept')

In [8]:
df_by_dept['amount'].describe()

count    44947.000000
mean        43.285574
std         46.598570
min          1.000000
25%         20.351389
50%         34.427007
75%         49.684070
max        561.666667
Name: amount, dtype: float64

## Label Encoder (Though I found it should not be used and deleted it later)

In [6]:
def label_preprocess(X, Y, column_list):
    X_train, X_test, y_train, y_test = train_test_split(
        X,Y,
        test_size = 0.2,
        random_state = 42
    )
    for i in range(len(column_list)):
        label_encoder = LabelEncoder()
        column_name = column_list[i]
        column_encoded = column_list[i]+'_encoded'
        X_train[column_encoded] = label_encoder.fit_transform(X_train[column_name])
        X_test[column_encoded] = label_encoder.transform(X_test[column_name])
        X_train.drop([column_name], axis = 1, inplace = True)
        X_test.drop([column_name], axis = 1, inplace = True)
    return X_train, X_test, y_train, y_test

In [17]:
X = df_by_dept.drop(['amount'], axis = 1)
Y = df_by_dept['amount']
X_train_label, X_test_label, y_train_label, y_test_label = label_preprocess(X,Y,['dept', 'saletype'])

## One-hot Encoder

In [12]:
def preprocess_onehot(X, Y, column_list):
    X_train, X_test, y_train, y_test = train_test_split(
        X,Y,
        test_size = 0.2,
        random_state = 42
    )
    onehot_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False)
    train_encoded = onehot_encoder.fit_transform(X_train[column_list])
    test_encoded = onehot_encoder.transform(X_test[column_list])
    encoded_columns = onehot_encoder.get_feature_names_out(column_list)
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoded_columns, index=X_train.index)
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoded_columns, index=X_test.index)
    X_train = pd.concat([X_train, train_encoded_df], axis = 1)
    X_test = pd.concat([X_test, test_encoded_df], axis = 1)
    X_train.drop(column_list, axis=1, inplace=True)
    X_test.drop(column_list, axis=1, inplace=True)
    return X_train, X_test, y_train, y_test

In [25]:
X_train_hot, X_test_hot, y_train_hot, y_test_hot = preprocess_onehot(X,Y,['dept', 'saletype'])

# Random Forest
## Use GridSearchCV to perform cross validation on the train data and use the best parameter on the test data

In [13]:
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [14]:
def weighted_mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.sum(np.abs(y_true - y_pred))
    denominator = np.sum(np.abs(y_true))
    wmape = (numerator / denominator) * 100
    
    return wmape

## Put in some different error calculation method, including Score, Mean Absolute Percentage Error, which is a great choice for nonlinear model, Mean Absolute Error, and Root Mean Square Error

## Perform it for one-hot Encoding

In [26]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2', None]
}
rf = RandomForestRegressor(random_state=8889)
grid_search_dummy = GridSearchCV(estimator=rf, 
                         param_grid=param_grid,
                         cv=5,  # 5-fold cross validation
                         n_jobs=-1,  # use all CPU cores
                         scoring='neg_mean_squared_error')  
grid_search_dummy.fit(X_train_hot, y_train_hot)

# 5. Print results
print("Best parameters:", grid_search.best_params_)
print("Best score:", np.sqrt(-grid_search.best_score_))

/nfs/home/pin2118/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'max_depth': 20, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Best score: 10.607846965404445


In [27]:
best_model_dummy = grid_search_dummy.best_estimator_
test_score_dummy = best_model_dummy.score(X_test_hot, y_test_hot)
print("Test set R² score:", test_score)
y_pred = best_model_dummy.predict(X_test_hot)
mape = calculate_mape(y_test_hot, y_pred)
mae = mean_absolute_error(y_test_hot, y_pred)
mse = np.mean((y_test_hot - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train_hot.columns,
    'importance': best_model_dummy.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))

Test set R² score: 0.9390277224102208
Mean Absolute Percentage Error (MAPE): 9.79%
Mean Absolute Error (MAE): 4.13
Root Mean Square Error (RMSE): 11.49

Feature importance:
        feature    importance
3          cost  4.941080e-01
4        retail  3.518676e-01
22    dept_3105  8.915119e-02
0    sale_month  3.119257e-02
2   sale_by_day  1.618470e-02
..          ...           ...
31    dept_4704  4.727178e-06
24    dept_3404  3.163891e-06
39    dept_5404  1.563772e-06
44    dept_6402  6.846969e-07
9     dept_1202  3.802534e-07

[67 rows x 2 columns]


# No difference between these two encoding methods

# 2: Number of sales by department prediction

# Extract the number of sales data

In [77]:
num_by_dept = df.loc[:,['dept','amount', 'saledate', 'saletype']].groupby(['dept', 'saledate','saletype']).count().reset_index()
num_by_dept.rename(columns = {'amount':'number_of_sales'}, inplace = True)
num_by_dept.loc[:,"number_of_sales"] = num_by_dept.loc[:,"number_of_sales"].astype(float)
num_by_dept.loc[:,"log_of_number_of_sales"] = np.log2(num_by_dept.loc[:,"number_of_sales"])

In [81]:
num_by_dept['saledate'] = pd.to_datetime(num_by_dept['saledate'])
num_by_dept['sale_month'] = num_by_dept['saledate'].dt.month
num_by_dept['sale_by_weekday'] = num_by_dept['saledate'].dt.weekday
num_by_dept['sale_by_day'] = num_by_dept['saledate'].dt.day
num_by_dept = num_by_dept.drop(['saledate'], axis = 1)

In [32]:
cost_retail_dept = df.groupby('dept')[['cost','retail']].mean().reset_index()
num_by_dept = pd.merge(num_by_dept, cost_retail_dept, on = 'dept')
num_by_dept = num_by_dept.drop(['saledate'], axis = 1)
num_by_dept.head()

,dept,saletype,number_of_sales,log_of_number_of_sales,sale_month,sale_by_weekday,sale_by_day,cost,retail
0,800,P,9060,13.145295,8,6,1,11.109902,18.517747
1,800,R,276,8.108524,8,6,1,11.109902,18.517747
2,800,P,10091,13.300782,8,0,2,11.109902,18.517747
3,800,R,356,8.475733,8,0,2,11.109902,18.517747
4,800,P,17424,14.088788,8,1,3,11.109902,18.517747


## Split it into training and testing, then apply label encoding and one-hot encoding separately

In [82]:
X = num_by_dept.drop(['number_of_sales', 'log_of_number_of_sales'], axis = 1)
y = num_by_dept['log_of_number_of_sales']
X_num_train_hot, X_num_test_hot, y_num_train_hot, y_num_test_hot = preprocess_onehot(X,y,['dept', 'saletype'])

# One-hot Encoding Random Forest Number Of Sales

In [149]:
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [10, 20, 25],
    'min_samples_split': [ 15,20, 25],
    'min_samples_leaf': [ 6, 8, 10],
    'max_features': [None]
}
rf = RandomForestRegressor(random_state=8889)
grid_search_dummy_num = GridSearchCV(estimator=rf, 
                         param_grid=param_grid,
                         cv=5,  # 5-fold cross validation
                         n_jobs=-1,  # use all CPU cores
                         scoring='neg_mean_squared_error')  
grid_search_dummy_num.fit(X_num_train_hot, y_num_train_hot)

# 5. Print results
print("Best parameters:", grid_search_dummy_num.best_params_)
print("Best score:", np.sqrt(-grid_search_dummy_num.best_score_))

Best parameters: {'max_depth': 25, 'max_features': None, 'min_samples_leaf': 6, 'min_samples_split': 20, 'n_estimators': 400}
Best score: 0.5859711035967916


In [150]:
best_model_num = grid_search_dummy_num.best_estimator_
test_score_num = best_model_num.score(X_num_test_hot, y_num_test_hot)
print("Test set Model score:", test_score)
y_pred = np.exp(best_model_num.predict(X_num_test_hot))
y_test = np.exp(y_num_test_hot)
mape = calculate_mape(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f" Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_num_train_hot.columns,
    'importance': best_model_num.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))

Test set Model score: 0.9062948775965912
Mean Absolute Percentage Error (MAPE): 47.42%
Mean Absolute Error (MAE): 212.35
 Mean Square Error (RMSE): 890.51

Feature importance:
       feature  importance
65  saletype_P    0.376107
4       retail    0.179344
0   sale_month    0.110810
3         cost    0.089737
5     dept_800    0.019601
..         ...         ...
8    dept_1107    0.000143
7    dept_1100    0.000083
17   dept_2301    0.000057
48   dept_7101    0.000051
10   dept_1301    0.000036

[67 rows x 2 columns]


# Xgboost for Number of Sales using one-hot encoding

In [88]:
param_grid = {
    'max_depth': [  5,7, 9],
    'learning_rate': [ 0.1, 0.15],
    'n_estimators': [ 400,500,600,700],
    'min_child_weight': [ 0.25,0.5, 1],
    'subsample': [ 1]
}
xgb_model = XGBRegressor(objective = 'reg:squarederror', seed = 8889)
grid_search_xgboost = GridSearchCV(
    estimator = xgb_model,
    cv = 5,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1)
grid_search_xgboost.fit(X_num_train_hot, y_num_train_hot)
print("Best parameters:", grid_search_xgboost.best_params_)
print("Best Scores:", np.sqrt(-grid_search_xgboost.best_score_))

Best parameters: {'learning_rate': 0.15, 'max_depth': 7, 'min_child_weight': 0.25, 'n_estimators': 600, 'subsample': 1}
Best Scores: 0.6225932969833398


In [89]:
best_model_xgboost = grid_search_xgboost.best_estimator_
test_score = best_model_xgboost.score(X_num_test_hot, y_num_test_hot)
print("Test set Model score:", test_score)
y_pred = np.power(2, best_model_xgboost.predict(X_num_test_hot))
y_test = np.power(2, y_num_test_hot)
mape = calculate_mape(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f" Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_num_train_hot.columns,
    'importance': best_model_xgboost.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))
#0.9574751, 29.1 171.88, 1187.39
# 0.9566507, 29.32, 163.91, 1031.70

Test set Model score: 0.951999387929494
Mean Absolute Percentage Error (MAPE): 32.56%
Mean Absolute Error (MAE): 169.40
 Mean Square Error (RMSE): 935.64

Feature importance:
            feature  importance
7         dept_1202    0.085684
3          dept_800    0.053720
38        dept_5506    0.051180
26        dept_4402    0.048988
25        dept_4400    0.047346
..              ...         ...
1   sale_by_weekday    0.000693
57        dept_8305    0.000561
2       sale_by_day    0.000491
47        dept_7102    0.000167
64       saletype_R    0.000000

[65 rows x 2 columns]


# PCA is experimented, though not working. Possibly due to lack of intersection and correlation between variables

In [14]:
scaler = StandardScaler()
scaled_dummy = scaler.fit_transform(num_by_dept[['dept_encoded', 'type_encoded', 'sale_month', 'sale_by_weekday','cost','retail', 'sale_by_day']])
scaled_dummy

array([[-1.67352341, -0.99469669,  0.42501525, ..., -0.47484755,
        -0.34034078, -1.66581653],
       [-1.67352341,  1.00533159,  0.42501525, ..., -0.47484755,
        -0.34034078, -1.66581653],
       [-1.67352341, -0.99469669,  0.42501525, ..., -0.47484755,
        -0.34034078, -1.55125054],
       ...,
       [ 1.74124386,  1.00533159,  0.42501525, ..., -0.09255401,
        -0.1878701 ,  1.19833316],
       [ 1.74124386, -0.99469669,  0.42501525, ..., -0.09255401,
        -0.1878701 ,  1.31289915],
       [ 1.74124386,  1.00533159,  0.42501525, ..., -0.09255401,
        -0.1878701 ,  1.31289915]])

In [37]:
dept_cols = dummy_vars.drop(['saledate', 'number_of_sales', 'log_of_number_of_sales','sqrt_number_of_sales'], axis = 1)
scaler = StandardScaler()
scaled_dept = scaler.fit_transform(dept_cols)
pca = PCA(n_components=40)
dept_pca = pca.fit_transform(scaled_dept)

In [38]:
dept_cols

,sale_month,sale_by_weekday,sale_by_day,cost,retail,dept_800,dept_801,dept_1100,dept_1107,dept_1202,...,dept_8101,dept_8104,dept_8305,dept_8306,dept_9000,dept_9105,dept_9306,dept_9801,saletype_P,saletype_R
0,8,6,1,11.109902,18.517747,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,8,6,1,11.109902,18.517747,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,8,0,2,11.109902,18.517747,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,8,0,2,11.109902,18.517747,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,8,1,3,11.109902,18.517747,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44942,8,3,25,22.001678,25.465765,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
44943,8,4,26,22.001678,25.465765,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
44944,8,4,26,22.001678,25.465765,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
44945,8,5,27,22.001678,25.465765,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


In [40]:
print(np.cumsum(pca.explained_variance_ratio_))

[0.04450736 0.0743658  0.09008587 0.10563443 0.12083207 0.13602035
 0.15120862 0.16639689 0.18158516 0.19677343 0.2119617  0.22714997
 0.24233824 0.25752651 0.27271479 0.28790306 0.30309133 0.3182796
 0.33346787 0.34865614 0.36384441 0.37903268 0.39422095 0.40940923
 0.4245975  0.43978577 0.45497404 0.47016231 0.48535058 0.50053885
 0.51572712 0.53091539 0.54610367 0.56129194 0.57648021 0.59166848
 0.60685675 0.62204502 0.63723329 0.65242156]


# In the end, Final Model! We want to predict monthly average store sales for each department with respect to cities in the US

In [7]:
df['saledate'] = pd.to_datetime(df['saledate'])
df['sale_month'] = df['saledate'].dt.month
grouped = df.groupby(['city', 'sale_month', 'saletype', 'store', 'dept']).size().reset_index(name='sales_count')
# Calculate the average number of sales per store for each city, saledate, and saletype
average_sales = grouped.groupby(['city', 'sale_month', 'saletype', 'dept']).mean().reset_index()
average_sales['log_sales_count'] = np.log(average_sales['sales_count'])
average_sales.drop(['store'], axis = 1, inplace = True)


In [8]:
cost_retail_grouped = df.groupby(['city', 'store', 'dept'])[['cost','retail']].mean().reset_index()
cost_retail_city = cost_retail_grouped.groupby(['city', 'dept']).mean().reset_index()
cost_retail_city.head()
average_sales_final = pd.merge(average_sales, cost_retail_city, on = ['city', 'dept'])

In [9]:
average_sales_final

,city,sale_month,saletype,dept,sales_count,log_sales_count,store,cost,retail
0,ABILENE,1,P,800,1268.0,7.145196,4407.0,10.978720,18.301210
1,ABILENE,1,P,801,3.0,1.098612,4407.0,72.310415,59.574273
2,ABILENE,1,P,1100,16.0,2.772589,4407.0,26.821457,43.667004
3,ABILENE,1,P,1107,68.0,4.219508,4407.0,17.801556,33.607459
4,ABILENE,1,P,1202,469.0,6.150603,4407.0,2.371473,5.038392
...,...,...,...,...,...,...,...,...,...
314725,YUMA,12,R,8305,27.0,3.295837,1009.0,21.064933,22.393650
314726,YUMA,12,R,8306,10.0,2.302585,1009.0,15.987684,23.120169
314727,YUMA,12,R,9000,1.0,0.000000,1009.0,13.303922,21.861111
314728,YUMA,12,R,9105,6.0,1.791759,1009.0,16.236871,20.632500


## We try two ways to process our response variable: Log, Square Root, and no process.

In [15]:
X = average_sales_final.drop(['sales_count', 'log_sales_count'], axis = 1)
y = average_sales_final['log_sales_count']
X_city_train_hot, X_city_test_hot, y_city_train_hot, y_city_test_hot = preprocess_onehot(X, y, ['city','dept', 'saletype'])

In [16]:
param_grid = {
    'max_depth': [  7, 9,11],
    'learning_rate': [  0.1,0.2,0.3],
    'n_estimators': [  2000, 2200, 2500],
    'min_child_weight': [ 5,15, 25],
    'subsample': [ 1]
}
xgb_model = XGBRegressor(objective = 'reg:squarederror', seed = 8889)
grid_search_xgboost_log = GridSearchCV(
    estimator = xgb_model,
    cv = 5,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1)
grid_search_xgboost_log.fit(X_city_train_hot, y_city_train_hot)
print("Best parameters:", grid_search_xgboost_log.best_params_)
print("Best Scores:", np.sqrt(-grid_search_xgboost_log.best_score_))

/nfs/home/pin2118/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_child_weight': 15, 'n_estimators': 2500, 'subsample': 1}
Best Scores: 0.46524758917239695


In [18]:
best_model_log = grid_search_xgboost_log.best_estimator_
test_score = best_model_log.score(X_city_test_hot, y_city_test_hot)
print("Test set model score:", test_score)
y_pred = np.exp(best_model_log.predict(X_city_test_hot))
y_test = np.exp(y_city_test_hot)
mape = weighted_mape(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_city_train_hot.columns,
    'importance': best_model_log.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))
# 0.2, 11, 25, 1500, 1, 0.92925, 38.1, 16.95, 62.49

Test set model score: 0.9297189250923495
Mean Absolute Percentage Error (MAPE): 20.08%
Mean Absolute Error (MAE): 16.90
Root Mean Square Error (RMSE): 63.43

Feature importance:
           feature  importance
320      dept_8104    0.064699
271      dept_1202    0.058439
293      dept_4704    0.051085
267       dept_800    0.040569
275      dept_2102    0.038769
..             ...         ...
256      city_WACO    0.000150
128  city_LAKELAND    0.000127
230  city_ST LOUIS    0.000000
50   city_CHEYENNE    0.000000
328     saletype_R    0.000000

[329 rows x 2 columns]


In [37]:
X = average_sales_final.drop(['sales_count', 'log_sales_count'], axis = 1)
y = average_sales_final['sales_count']
X_city_train_hot, X_city_test_hot, y_city_train_hot, y_city_test_hot = preprocess_onehot(X, y, ['city','dept', 'saletype'])

In [38]:
y.describe()

count    314730.000000
mean         84.591650
std         199.772551
min           1.000000
25%           6.000000
50%          20.333333
75%          79.000000
max        7451.000000
Name: sales_count, dtype: float64

In [44]:
plt.boxplot(y, vert = 0)
plt.title('Distribution of Response variable')
plt.ylabel('Number of Sales of each store on city and department level)
plt.show()


SyntaxError: unterminated string literal (detected at line 3) (928429321.py, line 3)

In [22]:
X_city_train_hot

,sale_month,store,cost,retail,city_ABILENE,city_AIKEN,city_AKRON,city_ALBANY,city_ALBUQUERQUE,city_ALEXANDRIA,...,dept_8101,dept_8104,dept_8305,dept_8306,dept_9000,dept_9105,dept_9306,dept_9801,saletype_P,saletype_R
12277,1,1407.0,10.013850,13.956912,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
56173,1,5323.0,17.893637,25.689160,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
249728,6,3403.0,15.872118,12.128471,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
172973,9,4702.0,17.692755,23.616068,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
138360,1,2303.0,5.242506,6.744543,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,8,4604.0,19.518774,27.637679,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
259178,3,209.0,9.260647,12.390782,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
131932,9,2303.0,17.764693,18.042473,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
146867,2,1702.0,30.878760,31.050435,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [53]:
param_grid = {
    'max_depth': [   11, 15],
    'learning_rate': [  0.1,0.2,0.3],
    'n_estimators': [  700, 1100, 1500],
    'min_child_weight': [ 15, 25, 35],
    'subsample': [ 1]
}
xgb_model = XGBRegressor(objective = 'reg:squarederror', seed = 8889)
grid_search_xgboost = GridSearchCV(
    estimator = xgb_model,
    cv = 5,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1, 
    verbose = 2)
grid_search_xgboost.fit(X_city_train_hot, y_city_train_hot)
print("Best parameters:", grid_search_xgboost.best_params_)
print("Best Scores:", np.sqrt(-grid_search_xgboost.best_score_))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 15, 'min_child_weight': 35, 'n_estimators': 1100, 'subsample': 1}
Best Scores: 74.06342036140617


In [70]:
best_model = grid_search_xgboost.best_estimator_
test_score = best_model.score(X_city_test_hot, y_city_test_hot)
print("Test set model score:", test_score)
y_pred = best_model.predict(X_city_test_hot)
y_test = y_city_test_hot
mape = weighted_mape(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_city_train_hot.columns,
    'importance': best_model.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))

Test set model score: 0.8867959259249272
Mean Absolute Percentage Error (MAPE): 26.99%
Mean Absolute Error (MAE): 22.72
Root Mean Square Error (RMSE): 66.87

Feature importance:
           feature  importance
267       dept_800    0.212321
275      dept_2102    0.109044
271      dept_1202    0.059851
290      dept_4402    0.032093
278      dept_2200    0.031113
..             ...         ...
225   city_SLIDELL    0.000040
70     city_DOTHAN    0.000030
50   city_CHEYENNE    0.000000
230  city_ST LOUIS    0.000000
328     saletype_R    0.000000

[329 rows x 2 columns]


In [19]:
X_sqrt = average_sales_final.drop(['sales_count', 'log_sales_count'], axis = 1)
y_sqrt = np.sqrt(average_sales_final['sales_count'])
X_city_train_sqrt, X_city_test_sqrt, y_city_train_sqrt, y_city_test_sqrt = preprocess_onehot(X_sqrt, y_sqrt, ['city','dept', 'saletype'])

In [20]:
param_grid = {
    'max_depth': [   11, 15, 19],
    'learning_rate': [  0.1,0.2,0.3],
    'n_estimators': [   1500, 1800, 2100],
    'min_child_weight': [ 15, 25, 35],
    'subsample': [ 1]
}
xgb_model = XGBRegressor(objective = 'reg:squarederror', seed = 8889)
grid_search_xgboost_sqrt = GridSearchCV(
    estimator = xgb_model,
    cv = 5,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1)
grid_search_xgboost_sqrt.fit(X_city_train_sqrt, y_city_train_sqrt)
print("Best parameters:", grid_search_xgboost_sqrt.best_params_)
print("Best Scores:", np.sqrt(-grid_search_xgboost_sqrt.best_score_))

/nfs/home/pin2118/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'learning_rate': 0.2, 'max_depth': 11, 'min_child_weight': 15, 'n_estimators': 2100, 'subsample': 1}
Best Scores: 1.3730120668436196


In [21]:
best_model_sqrt = grid_search_xgboost_sqrt.best_estimator_
test_score = best_model_sqrt.score(X_city_test_sqrt, y_city_test_sqrt)
print("Test set model score:", test_score)
y_pred = np.square(best_model_sqrt.predict(X_city_test_sqrt))
y_test = np.square(y_city_test_sqrt)
mape = weighted_mape(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
# 7. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_city_train_sqrt.columns,
    'importance': best_model_sqrt.feature_importances_
})
print("\nFeature importance:")
print(feature_importance.sort_values('importance', ascending=False))

Test set model score: 0.9579480325024534
Mean Absolute Percentage Error (MAPE): 19.48%
Mean Absolute Error (MAE): 16.39
Root Mean Square Error (RMSE): 59.39

Feature importance:
                   feature  importance
271              dept_1202    0.090771
320              dept_8104    0.079874
267               dept_800    0.068765
275              dept_2102    0.059792
293              dept_4704    0.057507
..                     ...         ...
57   city_COLORADO SPRINGS    0.000084
125      city_LAKE CHARLES    0.000071
50           city_CHEYENNE    0.000000
230          city_ST LOUIS    0.000000
328             saletype_R    0.000000

[329 rows x 2 columns]
